# Black branes in Lifshitz-like spacetimes

This SageMath notebook implements some computations of the article
- I. Ya. Aref'eva, A. A. Golubtsova & E. Gourgoulhon: *Analytic black branes in 
  Lifshitz-like backgrounds and thermalization*, [J. High Energ. Phys. 09(2016), 142](https://doi.org/10.1007/JHEP09(2016)142)

These computations are making use of differential geometry tools developed through the [SageManifolds](http://sagemanifolds.obspm.fr) project.

*NB:* a version of SageMath at least equal to 8.2 is required to run this notebook:

In [ ]:
version()

First we set up the notebook to display mathematical objects using LaTeX formatting:

In [ ]:
%display latex

## Five-dimensional Lifshitz-like spacetime

Let us declare the spacetime $M$ as a 5-dimensional manifold:

In [ ]:
M = Manifold(5, 'M', structure='Lorentzian')
print(M)

We introduce a first coordinate system on $M$:

In [ ]:
X.<t,x,y1,y2,r> = M.chart('t x y1:y_1 y2:y_2 r')
X

In [ ]:
g = M.metric()
ff = function('f')(r)
var('nu', latex_name=r'\nu', domain='real')
g[0,0] = -ff*exp(2*nu*r)
g[1,1] = exp(2*nu*r)
g[2,2] = exp(2*r)
g[3,3] = exp(2*r)
g[4,4] = 1/ff
g.display()

A matrix view of the metric components:

In [ ]:
g[:]

A view component by component (by default, only the nonzero ones are displayed):

In [ ]:
g.display_comp()

Access to a particular component:

In [ ]:
g[0,0]

If $f(r)=1$, this metric is invariant under the *Lifshitz scaling*
$$ (t,x,y_1,y_2,e^r) \longmapsto \left(\lambda^\nu t, \lambda^\nu x, \lambda y_1, \lambda y_2, \frac{e^r}{\lambda} \right)$$
- If $\nu=1$ the scaling is isotropic and we recognize the metric 
  of $\mathrm{AdS}_5$ in Poincaré coordinates ($M$ is then the Poincaré patch of $\mathrm{AdS}_5$)
- If $\nu\not=1$, the scaling is anisotropic

### Curvature

The Riemann tensor is

In [ ]:
Riem = g.riemann()
print(Riem)

In [ ]:
Riem.display_comp(only_nonredundant=True)

The Ricci tensor:

In [ ]:
Ric = g.ricci()
print(Ric)

In [ ]:
Ric.display()

In [ ]:
Ric.display_comp()

The Ricci scalar:

In [ ]:
Rscal = g.ricci_scalar()
print(Rscal)

In [ ]:
Rscal.display()

### Source model
Let us consider a model based on the following action, involving a dilaton scalar field $\phi$ and a Maxwell 2-form $F$:

$$ S = \int \left( R(g) - 2 \Lambda - \frac{1}{2} \nabla_m \phi \nabla^m \phi - \frac{1}{4} e^{\lambda\phi} F_{mn} F^{mn} \right) \sqrt{-g} \, \mathrm{d}^5 x \qquad\qquad \mbox{(1)}$$

where $R(g)$ is the Ricci scalar of metric $g$, $\Lambda$ is the cosmological constant and $\lambda$ is the dilatonic coupling constant.

### The dilaton scalar field

We consider the following ansatz for the dilaton scalar field $\phi$:
$$ \phi = \frac{1}{\lambda} \left( 4 r + \ln\mu \right),$$
where $\mu$ is a constant. 

In [ ]:
var('mu', latex_name=r'\mu', domain='real')
var('lamb', latex_name=r'\lambda', domain='real')
phi = M.scalar_field({X: (4*r + ln(mu))/lamb}, 
                     name='phi', latex_name=r'\phi')
phi.display()

The 1-form $\mathrm{d}\phi$ is

In [ ]:
dphi = phi.differential()
print(dphi)

In [ ]:
dphi.display()

In [ ]:
dphi[:]  # all the components in the default frame

### The 2-form field

We consider the following ansatz for $F$:
$$ F = \frac{1}{2} q \, \mathrm{d}y_1\wedge \mathrm{d}y_2, $$
where $q$ is a constant. 

Let us first get the 1-forms $\mathrm{d}y_1$ and $\mathrm{d}y_2$:

In [ ]:
X.coframe()

In [ ]:
dy1 = X.coframe()[2]
dy2 = X.coframe()[3]
print(dy1)
print(dy2)
dy1, dy2

Then we can form $F$ according to the above ansatz:

In [ ]:
var('q', domain='real')
F = q/2 * dy1.wedge(dy2)
F.set_name('F')
print(F)
F.display()

By construction, the 2-form $F$ is closed (since $q$ is constant):

In [ ]:
print(F.exterior_derivative())

In [ ]:
F.exterior_derivative().display()

Let us evaluate the square $F_{mn} F^{mn}$ of $F$:

In [ ]:
Fu = F.up(g)
print(Fu)
Fu.display()

In [ ]:
F2 = F['_{mn}']*Fu['^{mn}']  # using LaTeX notations to denote contraction
print(F2)
F2.display()

We shall also need the tensor $\mathcal{F}_{mn} := F_{mp} F_n^{\ \, p}$:

In [ ]:
FF = F['_mp'] * F.up(g,1)['^p_n']
print(FF)
FF.display()

The tensor field $\mathcal{F}$ is symmetric:

In [ ]:
FF == FF.symmetrize()

Therefore, from now on, we set

In [ ]:
FF = FF.symmetrize()

### Einstein equation

Let us first introduce the cosmological constant:

In [ ]:
var('Lamb', latex_name=r'\Lambda', domain='real')

From the action (1), the field equation for the metric $g$ is
$$ R_{mn} - \frac{2}{3}\Lambda \, g  - \frac{1}{2}\partial_m\phi \partial_n\phi -\frac{1}{2} e^{\lambda\phi} F_{mp} F^{\ \, p}_n + \frac{1}{12} e^{\lambda\phi} F_{rs} F^{rs} \, g_{mn} = 0 $$
We write it as

    EE == 0

with `EE` defined by

In [ ]:
EE = Ric - 2*Lamb/3*g - 1/2* (dphi*dphi) -  1/2*exp(lamb*phi)*FF \
     + 1/12*exp(lamb*phi)*F2*g
EE.set_name('E')
print(EE)

In [ ]:
EE.display_comp(only_nonredundant=True)

We note that `EE==0` leads to only 4 independent equations because

In [ ]:
EE[2,2] == EE[3,3]

The 4 independent equations are chosen to be

In [ ]:
eq0 = EE[0,0]/exp(2*nu*r)
eq0

In [ ]:
eq1 = EE[1,1]/exp(2*nu*r)
eq1

In [ ]:
eq2 = EE[2,2]/exp(2*r)
eq2

In [ ]:
eq3 = EE[4,4]*lamb^2*f(r)
eq3

### Dilaton field equation

First we evaluate $\nabla_m \nabla^m \phi$:

In [ ]:
box_phi = phi.dalembertian()
print(box_phi)
box_phi.display()

From the action (1), the field equation for $\phi$ is 
$$ \nabla_m \nabla^m \phi = \frac{\lambda}{4} e^{\lambda\phi} F_{mn} F^{mn}$$
We write it as

    DE == 0
    
with `DE` defined by

In [ ]:
DE = box_phi - lamb/4*exp(lamb*phi) * F2
print(DE)

In [ ]:
DE.display()

Hence the dilaton field equation provides a fith equation:

In [ ]:
eq4 = DE.coord_function()*lamb
eq4

### Maxwell equation

From the action (1), the field equation for $F$ is 
$$ \nabla_m \left( e^{\lambda\phi} F^{mn} \right)= 0 $$
To form it, we need the Levi-Civita connection $\nabla$ associated to $g$:

In [ ]:
nabla = g.connection()
print(nabla)
nabla

We write then the field equation for $F$ as

    ME == 0
    
with `ME` defined by

In [ ]:
ME = nabla(exp(lamb*phi)*Fu).trace(0,2)
print(ME)
ME.display()

We get identically zero.

### The solution

The Einstein equation + the dilaton field equation yields a system of 5 equations (eq0, eq1, eq2, eq3, eq4). 

Let us show that a solution is obtained for $\nu=2$ and $\nu=4$ with the following specific form of the blackening function:

$$ f(r) = 1 - m e^{-(2\nu +2)r}, $$

where $m$ is a constant. 

To this aim, we declare

In [ ]:
var('m', domain='real')
fm(r) = 1 - m*exp(-(2*nu+2)*r)
fm

and substitute this function for $f(r)$ in all the equations:

In [ ]:
eq0m = eq0.expr().substitute_function(f, fm).simplify_full()
eq0m

In [ ]:
eq0m = (eq0m * exp(2*nu*r+2*r)).simplify_full()
eq0m

In [ ]:
eq1m = eq1.expr().substitute_function(f, fm).simplify_full()
eq1m

In [ ]:
eq2m = eq2.expr().substitute_function(f, fm).simplify_full()
eq2m

In [ ]:
eq3m = eq3.expr().substitute_function(f, fm).simplify_full()
eq3m

In [ ]:
eq3m = (eq3m * exp(2*nu*r+2*r)).simplify_full()
eq3m

In [ ]:
eq4m = eq4.expr().substitute_function(f, fm).simplify_full()
eq4m

In [ ]:
eqs = [eq0m, eq1m, eq2m, eq3m, eq4m]

### Solution for $\nu = 2$

In [ ]:
neqs = [eq.subs(nu=2).simplify_full() for eq in eqs]
[eq == 0 for eq in neqs]

In [ ]:
solve([eq == 0 for eq in neqs], lamb, mu, Lamb)

Hence there are two families of solutions, each family being parametrized by e.g. $(q, m)$.

### Solution for $\nu=4$

In [ ]:
neqs = [eq.subs(nu=4).simplify_full() for eq in eqs]
[eq == 0 for eq in neqs]

In [ ]:
solve([eq == 0 for eq in neqs], lamb, mu, Lamb)

Again we get two families of solutions, each of them being parametrized by $(q, m)$.